In [76]:
import copy
import pandas as pd
import numpy as np
import time

from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
%matplotlib inline

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [5]:
%config IPCompleter.greedy=True

In [8]:
## URLS: get str seasons to plug into url
## for scraping the regular season per 100 possesion statistics 

seasons = [1996 + i for i in range(25)]
season_strs = [str(seasons[i]) + '-' + str(seasons[i+1])[2:] for i in range(len(seasons)-1)]
urls = ["https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=" + season + "&PerMode=Per100Possessions" for season in season_strs]
urls[0:5]


['https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=1996-97&PerMode=Per100Possessions',
 'https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=1997-98&PerMode=Per100Possessions',
 'https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=1998-99&PerMode=Per100Possessions',
 'https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=1999-00&PerMode=Per100Possessions',
 'https://stats.nba.com/players/traditional/?SeasonType=Regular%20Season&sort=PTS&dir=-1&Season=2000-01&PerMode=Per100Possessions']

In [141]:
def scrape_nba_stats(urls):
    
    for url in urls:
        browser = webdriver.Firefox(executable_path="./drivers/geckodriver")
        browser.get(url)
        time.sleep(5)
        
        ## take actions to get all data organized and on page
        sort_by_name(browser)
        time.sleep(5)
        select_all_pages(browser)
        time.sleep(5)
        raw_table = scrape_raw_table(browser)
        
        df = read_raw_stats_table(raw_table)
    
        
        
        return df
        


In [389]:
browser = webdriver.Firefox(executable_path="./drivers/geckodriver")
browser.get(urls[-1])

In [433]:
## Test funcs and see how to parse each table

# sort_by_name(browser)
# select_all_pages(browser)
# raw_table = scrape_raw_table(browser)

## FIX 
# df = read_raw_stats_table(raw_table)
# df.head()

select_different_stats_type(browser, 7)
# sort_by_name(browser)
# select_per_100(browser)
# select_all_pages(browser)

In [307]:
scoring_99 = scrape_raw_table(browser)
scoring_99 = scoring_99.text.split('\n')
scoring_99[0:31]

['PLAYER TEAM AGE GP W L MIN %FGA',
 '2PT %FGA',
 '3PT %PTS',
 '2PT %PTS',
 '2PT MR %PTS',
 '3PT %PTS',
 'FBPS %PTS',
 'FT %PTS',
 'OFFTO %PTS',
 'PITP 2FGM',
 '%AST 2FGM',
 '%UAST 3FGM',
 '%AST 3FGM',
 '%UAST FGM',
 '%AST FGM',
 '%UAST',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'A.C. Green']

In [305]:
# scoring_09 = scrape_raw_table(browser)
# scoring_09 = scoring_09.text.split('\n')
scoring_09[0:31]

['PLAYER TEAM AGE GP W L MIN %FGA',
 '2PT %FGA',
 '3PT %PTS',
 '2PT %PTS',
 '2PT MR %PTS',
 '3PT %PTS',
 'FBPS %PTS',
 'FT %PTS',
 'OFFTO %PTS',
 'PITP 2FGM',
 '%AST 2FGM',
 '%UAST 3FGM',
 '%AST 3FGM',
 '%UAST FGM',
 '%AST FGM',
 '%UAST',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'AJ Price']

In [304]:
# scoring_08 = scrape_raw_table(browser)
# scoring_08 = scoring_08.text.split('\n')
scoring_08[0:31]


['PLAYER TEAM AGE GP W L MIN %FGA',
 '2PT %FGA',
 '3PT %PTS',
 '2PT %PTS',
 '2PT MR %PTS',
 '3PT %PTS',
 'FBPS %PTS',
 'FT %PTS',
 'OFFTO %PTS',
 'PITP 2FGM',
 '%AST 2FGM',
 '%UAST 3FGM',
 '%AST 3FGM',
 '%UAST FGM',
 '%AST FGM',
 '%UAST',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Aaron Brooks']

In [434]:
raw_table = scrape_raw_table(browser)
df = read_raw_stats_table(raw_table, 7)
print(df.shape)
df.head()

(50, 19)


,TEAM,AGE,GP,W,L,MIN,DEF RTG,DREB,DREB%,%DREB,STL,STL%,BLK,%BLK,OPP PTS OFF TOV,OPP PTS 2ND CHANCE,OPP PTS FB,OPP PTS PAINT,DEF WS
A.C. Green,DAL,34,82,20,62,52.8,105.3,9.0,17.3,29.6,1.6,19.3,0.5,9.7,17.6,17.8,17.6,48.7,0.104
Aaron McKie,PHI,25,81,34,47,52.0,96.3,5.0,9.8,16.0,2.9,27.9,0.4,6.2,17.6,14.2,12.1,36.3,0.253
Aaron Williams,SEA,26,65,49,16,49.4,98.3,6.5,12.8,21.5,1.2,11.7,2.5,48.1,18.9,14.4,16.1,43.0,0.224
Adam Keefe,UTA,28,80,61,19,52.9,102.4,6.7,13.1,20.3,1.3,15.4,0.6,12.0,18.6,13.6,11.8,39.8,0.154
Adonal Foyle,GSW,23,55,9,46,50.2,99.5,8.5,15.7,26.9,1.0,13.7,4.0,54.2,20.6,17.9,15.4,42.0,0.196


In [364]:
raw_table = scrape_raw_table(browser)
df = read_raw_stats_table(raw_table, 7)
print(df.shape)
df.head()

table 0:5
['PLAYER TEAM AGE GP W L MIN USG% %FGM %FGA %3PM %3PA %FTM %FTA %OREB %DREB %REB %AST %TOV %STL %BLK %BLKA %PF %PFD %PTS', 'Aaron Gordon', 'ORL 24 62 30 32 2017.0 20.5 19.5 20.5 15.9 17.8 23.5 26.3 26.0 25.2 25.4 21.1 19.3 16.2 18.3 22.2 17.6 26.7 19.7', 'Aaron Holiday', 'IND 23 66 42 24 1617.0 18.2 16.3 18.8 23.4 22.9 15.4 14.1 7.4 11.5 10.7 25.7 21.0 22.8 9.0 17.8 18.5 18.1 16.9']
(501, 24)


,TEAM,AGE,GP,W,L,MIN,USG%,%FGM,%FGA,%3PM,...,%DREB,%REB,%AST,%TOV,%STL,%BLK,%BLKA,%PF,%PFD,%PTS
Aaron Gordon,ORL,24,62,30,32,2017.0,20.5,19.5,20.5,15.9,...,25.2,25.4,21.1,19.3,16.2,18.3,22.2,17.6,26.7,19.7
Aaron Holiday,IND,23,66,42,24,1617.0,18.2,16.3,18.8,23.4,...,11.5,10.7,25.7,21.0,22.8,9.0,17.8,18.5,18.1,16.9
Abdel Nader,OKC,26,55,37,18,867.0,16.4,18.0,17.1,28.9,...,14.2,13.5,9.5,16.6,17.7,19.6,13.3,22.5,13.5,18.6
Adam Mokoka,CHI,21,11,3,8,112.0,11.0,11.4,13.1,15.8,...,4.5,9.6,8.2,6.7,16.7,0.0,28.6,27.0,7.0,11.1
Admiral Schofield,WAS,23,33,9,24,368.0,11.8,11.8,13.4,20.9,...,17.0,15.3,8.3,6.7,14.5,10.5,11.8,28.9,11.2,12.2


In [415]:
df.tail()

,TEAM,AGE,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST%,...,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE
Brandon Goodwin,ATL,24,34,13,21,12.7,94.5,108.2,-13.8,20.8,...,18.0,2.3,13.2,7.6,10.2,46.8,50.7,22.0,104.54,9.4
Brandon Ingram,NOP,22,62,25,37,33.9,110.0,111.3,-1.3,19.2,...,15.3,2.3,14.4,8.4,11.2,53.1,58.7,27.5,105.01,13.0
Brandon Knight,DET,28,25,4,21,18.5,108.8,116.4,-7.6,21.6,...,23.8,0.9,9.3,4.7,13.1,48.8,50.3,20.7,98.80,5.2
Brook Lopez,MIL,32,68,53,15,26.7,112.2,100.1,12.0,7.3,...,10.9,3.1,11.8,7.7,7.7,51.1,55.1,18.1,106.60,8.8
Bruce Brown,DET,23,58,18,40,28.2,106.0,110.4,-4.3,20.8,...,27.7,3.8,13.1,8.4,12.0,48.0,51.8,15.9,98.00,8.3


In [293]:
# text = raw_table.text.split('\n')
text[16:31]

['', '', '', '', '', '', '', '', '', '', '', '', '', '', 'AJ Hammons']

In [218]:
trad_raw = scrape_raw_table(browser)
trad = trad_raw.text.split('\n')
trad[0:20]

## requires per 100

['PLAYER TEAM AGE GP W L MIN PTS FGM FGA FG% 3PM 3PA 3P% FTM FTA FT% OREB DREB REB AST TOV STL BLK PF FP DD2 TD3 +/-',
 'Aaron Gordon',
 'ORL 24 62 30 32 48.2 21.4 8.0 18.3 43.7 1.7 5.7 30.8 3.6 5.4 67.4 2.6 8.8 11.4 5.5 2.4 1.2 0.9 3.0 47.3 20 1 -1.6',
 'Aaron Holiday',
 'IND 23 66 42 24 47.6 18.5 6.9 16.6 41.4 2.6 6.5 39.4 2.2 2.6 85.1 0.6 3.9 4.6 6.6 2.6 1.6 0.5 3.5 37.6 3 0 3.3',
 'Abdel Nader',
 'OKC 26 55 37 18 47.2 18.8 6.7 14.3 46.8 2.6 7.0 37.5 2.8 3.6 77.3 0.8 4.7 5.4 2.1 2.3 1.3 1.1 4.3 33.1 0 0 -4.6',
 'Adam Mokoka',
 'CHI 21 11 3 8 46.9 13.4 5.0 11.8 42.9 2.5 6.3 40.0 0.8 1.7 50.0 2.9 1.3 4.2 1.7 0.8 1.7 0.0 7.1 25.2 0 0 21.0',
 'Admiral Schofield',
 'WAS 23 33 9 24 46.6 12.5 4.4 11.6 38.0 2.4 7.7 31.1 1.3 1.9 66.7 0.9 5.1 5.9 1.9 0.9 1.0 0.5 6.3 26.2 0 0 -7.2',
 'Al Horford',
 'PHI 34 67 39 28 47.8 18.8 7.5 16.7 45.0 2.3 6.7 35.0 1.4 1.9 76.3 2.4 8.3 10.8 6.4 1.9 1.2 1.4 3.3 47.4 6 0 3.0',
 'Al-Farouq Aminu',
 'ORL 29 18 7 11 47.7 9.8 3.1 10.8 29.1 1.1 4.5 25.0 2.4 3.6 65

In [221]:
adv_raw = scrape_raw_table(browser)
adv = adv_raw.text.split('\n')
adv[0:20]

['PLAYER',
 'TEAM AGE GP W L MIN OFFRTG DEFRTG NETRTG AST% AST/TO AST RATIO OREB% DREB% REB% TO RATIO EFG% TS% USG% PACE PIE',
 'Aaron Gordon',
 'ORL 24 62 30 32 32.5 108.6 109.7 -1.2 16.5 2.28 19.1 5.0 18.1 11.4 8.4 48.4 51.6 20.5 99.72 10.4',
 'Aaron Holiday',
 'IND 23 66 42 24 24.5 109.1 106.9 2.2 18.8 2.56 24.6 1.3 7.7 4.6 9.6 49.1 52.1 18.2 100.31 7.8',
 'Abdel Nader',
 'OKC 26 55 37 18 15.8 100.9 105.2 -4.2 6.8 0.88 10.3 1.6 9.5 5.6 11.6 55.9 59.1 16.4 101.79 7.2',
 'Adam Mokoka',
 'CHI 21 11 3 8 10.2 121.4 104.4 17.1 4.3 2.00 11.4 5.7 2.9 4.4 5.7 53.6 53.8 11.0 100.37 2.3',
 'Admiral Schofield',
 'WAS 23 33 9 24 11.2 102.9 110.0 -7.1 5.7 2.14 12.4 1.7 10.6 5.9 5.8 48.4 50.2 11.8 103.13 3.5',
 'Al Horford',
 'PHI 34 67 39 28 30.2 110.1 106.8 3.3 18.7 3.38 24.7 5.1 17.1 11.1 7.3 52.0 53.6 17.3 100.63 11.2',
 'Al-Farouq Aminu',
 'ORL 29 18 7 11 21.1 94.7 100.1 -5.4 8.8 1.24 15.2 5.3 15.8 10.2 12.3 34.3 39.5 12.7 100.15 6.3',
 'Alec Burks',
 'PHI 28 66 21 45 26.6 103.5 109.8 -6.3 17

In [225]:
misc_raw = scrape_raw_table(browser)
misc = misc_raw.text.split('\n')
misc[0:20]

## requires per 100misc

['PLAYER',
 'TEAM AGE GP W L MIN PTS',
 'OFF TO 2ND',
 'PTS FBPS PITP OPP PTS',
 'OFF TO OPP',
 '2ND PTS OPP',
 'FBPS OPP',
 'PITP BLK BLKA PF PFD',
 '',
 '',
 '',
 '',
 '',
 'Aaron Gordon',
 'ORL 24 62 30 32 48.2 3.8 3.6 3.1 10.9 13.7 11.1 11.3 46.3 0.9 1.0 3.0 4.9',
 'Aaron Holiday',
 'IND 23 66 42 24 47.6 2.6 1.9 2.3 6.7 14.0 13.8 14.0 46.5 0.5 0.8 3.5 3.4',
 'Abdel Nader',
 'OKC 26 55 37 18 47.2 3.5 2.0 3.0 7.5 16.3 11.5 10.8 47.4 1.1 0.5 4.3 2.7',
 'Adam Mokoka']

In [295]:
# scoring_raw = scrape_raw_table(browser)
# scoring = scoring_raw.text.split('\n')
scoring[16:31]


['%UAST', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [362]:
# usg_raw = scrape_raw_table(browser)
# usg = usg_raw.text.split('\n')
usg[0:20]


['PLAYER',
 'TEAM AGE GP W L MIN USG% %FGM %FGA %3PM %3PA %FTM %FTA %OREB %DREB %REB %AST %TOV %STL %BLK %BLKA %PF %PFD %PTS',
 'Aaron Gordon',
 'ORL 24 62 30 32 2017.0 20.5 19.5 20.5 15.9 17.8 23.5 26.3 26.0 25.2 25.4 21.1 19.3 16.2 18.3 22.2 17.6 26.7 19.7',
 'Aaron Holiday',
 'IND 23 66 42 24 1617.0 18.2 16.3 18.8 23.4 22.9 15.4 14.1 7.4 11.5 10.7 25.7 21.0 22.8 9.0 17.8 18.5 18.1 16.9',
 'Abdel Nader',
 'OKC 26 55 37 18 867.0 16.4 18.0 17.1 28.9 25.3 16.1 16.2 10.3 14.2 13.5 9.5 16.6 17.7 19.6 13.3 22.5 13.5 18.6',
 'Adam Mokoka',
 'CHI 21 11 3 8 112.0 11.0 11.4 13.1 15.8 18.1 4.9 6.6 18.9 4.5 9.6 8.2 6.7 16.7 0.0 28.6 27.0 7.0 11.1',
 'Admiral Schofield',
 'WAS 23 33 9 24 368.0 11.8 11.8 13.4 20.9 21.7 7.6 8.6 9.6 17.0 15.3 8.3 6.7 14.5 10.5 11.8 28.9 11.2 12.2',
 'Al Horford',
 'PHI 34 67 39 28 2026.0 17.3 18.1 19.2 19.5 20.9 9.6 9.4 25.2 23.5 23.9 23.5 13.3 15.5 29.8 14.8 16.7 11.6 17.1',
 'Al-Farouq Aminu',
 'ORL 29 18 7 11 380.0 12.7 9.5 12.4 11.5 13.4 12.8 15.0 23.8 22.3 22.7

In [237]:
# opp_raw = scrape_raw_table(browser)
# opp = opp_raw.text.split('\n')
opp[41]

## requires per 100

'Adams, Steven'

In [379]:
# defensive_raw = scrape_raw_table(browser)
# defensive = defensive_raw.text.split('\n')
defensive[0:20]

## requires per 100

['PLAYER',
 'TEAM AGE GP W L MIN DEF RTG DREB DREB% %DREB STL STL% BLK %BLK OPP PTS',
 'OFF TOV OPP PTS',
 '2ND CHANCE OPP PTS',
 'FB OPP PTS',
 'PAINT DEF',
 'WS',
 '',
 '',
 '',
 '',
 'Aaron Gordon',
 'ORL 24 62 30 32 48.2 109.7 8.8 18.1 25.2 1.2 16.2 0.9 18.3 13.7 11.1 11.3 46.3 0.130',
 'Aaron Holiday',
 'IND 23 66 42 24 47.6 106.9 3.9 7.7 11.5 1.6 22.8 0.5 9.0 14.0 13.8 14.0 46.5 0.167',
 'Abdel Nader',
 'OKC 26 55 37 18 47.2 105.2 4.7 9.5 14.2 1.3 17.7 1.1 19.6 16.3 11.5 10.8 47.4 0.195',
 'Adam Mokoka',
 'CHI 21 11 3 8 46.9 104.4 1.3 2.9 4.5 1.7 16.7 0.0 0.0 12.6 13.9 9.7 40.3 0.195',
 'Admiral Schofield']

In [421]:
## I need a way to deal with the varying header formats
## once i get to players it's a simple every other is index or stats untill player is empty

def read_raw_stats_table(raw_table, table_num):
    """
    table_number mappings
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    table = raw_table.text.split('\n')
    
    if table_num == 1:  # traditional
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-']
    
    elif table_num == 2:  # advanced
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'OFFRTG', 'DEFRTG', 'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TO RATIO', 'EFG%', 'TS%', 'USG%', 'PACE', 'PIE']
    
    elif table_num == 3: # misc
        cols = ['TEAM','AGE','GP','W','L','MIN','PTS OFF TO','2ND PTS','FBPS','PITP','OPP PTS OFF TO','OPP 2ND PTS','OPP FBPS','OPP PITP','BLK','BLKA','PF','PFD']

    elif table_num == 4: # scoring
        cols = ['TEAM', 'AGE', 'GP', 'W','L','MIN','%FGA 2PT','%FGA 3PT','%PTS 2PT','%PTS 2PT MR','%PTS 3PT','%PTS FBPS','%PTS FT','%PTS OFFTO','%PTS PITP','2FGM %AST','2FGM %UAST','3FGM %AST','3FGM %UAST','FGM %AST','FGM %UAST']
    
    elif table_num == 5:  # usage
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'USG%', '%FGM', '%FGA', '%3PM', '%3PA', '%FTM', '%FTA', '%OREB', '%DREB', '%REB', '%AST', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD', '%PTS']
    
#     ## MISSING DATA FROM PRIOR 2010'S
#     elif table_num == 6: # opponent
#         cols = ['TEAM','GP','W','L','MIN','OPP FGM','OPP FGA','OPP FG%','OPP 3PM','OPP 3PA','OPP 3P%','OPP FTM','OPP FTA','OPP FT%','OPP OREB','OPP DREB','OPP REB','OPP AST','OPP TOV','OPP STL','OPP BLK','OPP BLKA','OPP PF','OPP PFD','OPP PTS','+/-',]
    
    elif table_num == 7:
        cols = ['TEAM','AGE','GP','W','L','MIN','DEF RTG','DREB','DREB%','%DREB','STL','STL%','BLK','%BLK','OPP PTS OFF TOV','OPP PTS 2ND CHANCE','OPP PTS FB','OPP PTS PAINT','DEF WS']


    for i, line in enumerate(table):
        if line and line[0] == 'A' and len(line.split(' ')) == 2:
            body = table[i:] 
            break
            
    ## parse body of table now and extract index, stats
    index, all_stats = [], []
    for i in range(0, len(body), 2):
        player = body[i]
        if len(player.split(' ')) > 0 and len(player.split(' ')) < 3:
            index.append(player)
            stats = body[i+1].split(' ')
            all_stats.append(stats)
        
    return pd.DataFrame(all_stats, index=index, columns=cols)

In [395]:
a = ''
a[0]

IndexError: string index out of range

In [387]:
a = [55,22,33,66,77,88]
for i, n in enumerate(a):
    print(i, n)

0 55
1 22
2 33
3 66
4 77
5 88


In [337]:
a = ['a','b','c']
b = a[1:]
b[0] = 99
a


['a', 'b', 'c']

In [184]:
def sort_by_name(browser):
    xpath_player_sort = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/thead/tr/th[2]'
    browser.find_element_by_xpath(xpath_player_sort).click()
    browser.find_element_by_xpath(xpath_player_sort).click()
    return

def select_all_pages(browser):
    xpath_page_selection = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'
    xpath_all_pages = xpath_page_selection + '/option[1]'
    browser.find_element_by_xpath(xpath_all_pages).click()
    return
  
def scrape_raw_table(browser):
    xpath_stats_table = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]'
    return browser.find_element_by_xpath(xpath_stats_table)

def select_per_100(browser):
    xpath_per_mode = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select'
    browser.find_element_by_xpath(xpath_per_mode).click()
    
    xpath_per_100 = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select/option[3]'
    browser.find_element_by_xpath(xpath_per_100).click()
    

def select_different_stats_type(browser, n):
    """
    Different n values -->
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    assert n > 0 and n < 8
    ## select the header to get drop down
    xpath_stat_type_button = '/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/div/a'
    browser.find_element_by_xpath(xpath_stat_type_button).click()

    ## navigate to different stat type
    xpath_stat_type = f'/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/ul/li[{n}]/a/span'
    browser.find_element_by_xpath(xpath_stat_type).click()
    return


def select_seaons(browser):
    pass

In [136]:
df = scrape_nba_stats(urls)
df.head()

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
A.C. Green,DAL,33,83,23,60,53.1,12.7,5.0,10.3,48.3,...,14.0,1.5,1.6,1.5,0.3,3.1,35.6,16,0,-7.4
Aaron McKie,DET,24,83,48,35,52.7,14.0,4.9,11.8,41.1,...,7.2,5.2,2.9,2.5,0.7,4.2,37.2,1,0,4.3
Aaron Williams,VAN,25,33,4,29,53.0,19.1,8.0,13.9,57.4,...,13.5,1.4,3.0,1.5,2.7,6.8,47.1,0,0,-9.8
Acie Earl,MIL,27,47,14,33,50.7,19.0,6.8,18.1,37.4,...,9.7,2.0,3.5,1.5,2.8,6.2,43.3,0,0,-6.1
Adam Keefe,UTA,27,62,48,14,51.9,13.3,4.6,9.1,51.3,...,12.2,1.8,2.5,1.7,0.7,5.5,35.5,1,0,6.9


In [137]:
df.tail()

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
Willie Anderson,MIA,30,28,22,6,53.5,14.8,5.2,11.4,45.3,...,7.5,6.0,3.4,2.5,0.7,6.4,39.0,0,0,-2.3
Willie Burton,ATL,29,24,15,9,52.3,20.2,5.3,15.9,33.6,...,5.6,1.5,3.7,1.1,0.4,7.5,30.1,0,0,-7.8
Xavier McDaniel,NJN,34,62,19,43,49.4,14.6,5.8,14.9,38.9,...,13.4,2.7,2.9,1.5,0.7,6.1,38.5,4,0,0.2
Yinka Dare,NJN,24,41,12,29,48.6,8.8,2.9,8.3,35.2,...,12.7,0.5,3.2,0.6,4.3,7.9,36.3,0,0,0.8
Zan Tabak,TOR,27,13,6,7,51.6,19.9,7.6,16.8,45.1,...,11.6,3.3,5.0,1.4,2.6,8.3,45.9,0,0,-10.4


In [131]:
df['PTS'].sort_values()[0:10]

Bruce Bowen        0.0
Anthony Miller     0.0
Cuonzo Martin      0.0
Elmore Spencer     0.0
Derrick Alston     0.0
James Scott        1.5
Harvey Grant      10.0
Mike Brown        10.1
Duane Causwell    10.4
Greg Anderson     10.5
Name: PTS, dtype: object

In [127]:
## function for reading in the raw table scraped with selenium and returning DF

## need to make sure this works for the different types of stats

def read_raw_stats_table(raw_table):
    """Convert raw html element to DF"""
    table = raw_table.text.split('\n')
    cols = table[1].split(' ')
    index, all_stats = [], []
    
    for i in range(2, len(table[2:]), 2):
        player = table[i]
        if len(player.split(' ')) > 0 and len(player.split(' ')) < 3:
            index.append(player)
            stats = table[i+1].split(' ')
            all_stats.append(stats)
    
    df = pd.DataFrame(all_stats, index=index, columns=cols)
    
    return df
            

In [110]:
## Isolate player index
index = []
all_stats = []

for i in range(2, len(lines[2:]), 2):
#     print(lines[i])
    player = lines[i]
    if len(player.split(' ')) > 0 and len(player.split(' ')) < 3:
        index.append(player)
        stats = lines[i+1].split(' ')
        all_stats.append(stats)
    
    


    

In [116]:
df = pd.DataFrame(all_stats, index=index, columns=cols)
df.tail()

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
Willie Anderson,MIA,30,28,22,6,53.5,14.8,5.2,11.4,45.3,...,7.5,6.0,3.4,2.5,0.7,6.4,39.0,0,0,-2.3
Willie Burton,ATL,29,24,15,9,52.3,20.2,5.3,15.9,33.6,...,5.6,1.5,3.7,1.1,0.4,7.5,30.1,0,0,-7.8
Xavier McDaniel,NJN,34,62,19,43,49.4,14.6,5.8,14.9,38.9,...,13.4,2.7,2.9,1.5,0.7,6.1,38.5,4,0,0.2
Yinka Dare,NJN,24,41,12,29,48.6,8.8,2.9,8.3,35.2,...,12.7,0.5,3.2,0.6,4.3,7.9,36.3,0,0,0.8
Zan Tabak,TOR,27,13,6,7,51.6,19.9,7.6,16.8,45.1,...,11.6,3.3,5.0,1.4,2.6,8.3,45.9,0,0,-10.4


In [111]:
all_stats[0:5]

[['DAL',
  '33',
  '83',
  '23',
  '60',
  '53.1',
  '12.7',
  '5.0',
  '10.3',
  '48.3',
  '0.0',
  '0.4',
  '5.0',
  '2.7',
  '4.2',
  '65.0',
  '4.7',
  '9.2',
  '14.0',
  '1.5',
  '1.6',
  '1.5',
  '0.3',
  '3.1',
  '35.6',
  '16',
  '0',
  '-7.4'],
 ['DET',
  '24',
  '83',
  '48',
  '35',
  '52.7',
  '14.0',
  '4.9',
  '11.8',
  '41.1',
  '1.3',
  '3.3',
  '39.8',
  '3.0',
  '3.6',
  '83.6',
  '1.3',
  '5.9',
  '7.2',
  '5.2',
  '2.9',
  '2.5',
  '0.7',
  '4.2',
  '37.2',
  '1',
  '0',
  '4.3'],
 ['VAN',
  '25',
  '33',
  '4',
  '29',
  '53.0',
  '19.1',
  '8.0',
  '13.9',
  '57.4',
  '0.0',
  '0.1',
  '0.0',
  '3.1',
  '4.6',
  '67.3',
  '5.8',
  '7.6',
  '13.5',
  '1.4',
  '3.0',
  '1.5',
  '2.7',
  '6.8',
  '47.1',
  '0',
  '0',
  '-9.8'],
 ['MIL',
  '27',
  '47',
  '14',
  '33',
  '50.7',
  '19.0',
  '6.8',
  '18.1',
  '37.4',
  '0.0',
  '0.5',
  '0.0',
  '5.5',
  '8.5',
  '64.3',
  '3.5',
  '6.2',
  '9.7',
  '2.0',
  '3.5',
  '1.5',
  '2.8',
  '6.2',
  '43.3',
  '0',
  '0',
 

In [43]:

lines[0:10]
mj_stats = lines[3].split(' ')
print(len(mj_stats))
mj_stats

28


['CHI',
 '34',
 '82',
 '69',
 '13',
 '52.4',
 '41.0',
 '15.5',
 '31.9',
 '48.6',
 '1.9',
 '5.0',
 '37.4',
 '8.1',
 '9.7',
 '83.3',
 '1.9',
 '6.2',
 '8.1',
 '5.9',
 '2.8',
 '2.4',
 '0.7',
 '2.6',
 '66.2',
 '9',
 '1',
 '13.8']

In [86]:
lines[-1:-20:-1]
## ok, can read the lines and build DF
## if len line is 1, discard it (old index), if its a name then create new row in DF


['SAC 25 24 9 15 50.8 21.0 7.6 17.2 44.0 0.0 0.0 0.0 5.8 7.5 77.8 4.1 7.7 11.8 2.3 3.7 1.6 2.1 6.2 46.0 4 0 -8.1',
 'Brian Grant',
 'ORL 23 14 8 6 43.5 15.3 6.1 16.8 36.4 3.1 6.1 50.0 0.0 0.0 0.0 0.8 5.3 6.1 5.3 1.5 0.8 1.5 4.6 36.0 0 0 -9.9',
 'Brian Evans',
 'BOS 29 70 11 59 47.2 10.9 3.8 8.6 44.6 0.0 0.1 0.0 3.2 4.3 73.8 4.0 7.7 11.7 1.2 2.9 1.1 2.3 8.4 34.0 1 0 -12.1',
 'Brett Szabo',
 'IND 26 16 7 9 45.3 16.0 6.7 14.3 47.1 0.0 0.0 0.0 2.5 5.0 50.0 2.5 5.0 7.6 2.5 3.4 0.8 0.8 11.8 30.5 0 0 4.2',
 'Brent Scott',
 'HOU 28 25 14 11 50.4 16.2 5.7 13.5 41.9 2.2 6.8 32.1 2.7 2.7 100 1.3 2.4 3.7 8.4 4.1 2.2 0.0 4.4 35.7 0 0 4.3',
 'Brent Price',
 'LAC 25 59 24 35 50.2 20.2 7.1 17.4 40.9 2.6 7.9 32.4 3.5 4.3 81.7 1.4 3.7 5.0 7.1 3.5 2.3 0.7 4.0 42.4 1 0 4.6',
 'Brent Barry',
 'TOR 32 6 1 5 49.1 11.0 4.4 16.5 26.7 2.2 7.7 28.6 0.0 0.0 0.0 1.1 6.6 7.7 1.1 1.1 1.1 0.0 6.6 24.1 0 0 -34.1',
 'Brad Lohaus',
 'CLE 27 69 36 33 56.9 20.7 7.8 18.3 42.8 2.0 5.2 39.4 3.0 4.2 71.8 1.5 4.4 5.9 5.6 3.2 2

In [79]:
lines[0:20]

['PLAYER TEAM AGE GP W L MIN PTS FGM FGA FG% 3PM 3PA 3P% FTM FTA FT% OREB DREB REB AST TOV STL BLK PF FP DD2 TD3 +/-',
 '1',
 'Michael Jordan',
 'CHI 34 82 69 13 52.4 41.0 15.5 31.9 48.6 1.9 5.0 37.4 8.1 9.7 83.3 1.9 6.2 8.1 5.9 2.8 2.4 0.7 2.6 66.2 9 1 13.8',
 '2',
 'Karl Malone',
 'UTA 33 82 64 18 51.9 39.0 15.0 27.2 55.0 0.0 0.2 0.0 9.0 12.0 75.5 3.3 10.7 14.0 6.4 4.0 2.0 0.8 3.7 69.7 43 1 13.3',
 '3',
 'David Robinson',
 'SAS 31 6 3 3 52.5 37.9 12.9 25.7 50.0 0.0 0.0 0.0 12.1 18.6 65.4 6.8 11.4 18.2 2.9 2.9 2.1 2.1 3.2 74.0 2 0 7.5',
 '4',
 "Shaquille O'Neal",
 'LAL 25 51 38 13 52.3 36.0 14.9 26.7 55.7 0.0 0.1 0.0 6.2 12.9 48.4 5.2 12.0 17.2 4.3 3.9 1.2 4.0 4.8 74.7 44 0 7.2',
 '5',
 'Chris Gatling',
 'NJN 29 47 16 31 51.1 35.5 13.0 24.8 52.5 0.0 0.2 16.7 9.4 13.1 71.7 5.3 9.4 14.7 1.1 4.8 1.6 1.2 5.5 58.5 16 0 -3.5',
 '6',
 'Mitch Richmond',
 'SAC 32 81 34 47 51.4 35.2 12.0 26.5 45.4 3.4 8.0 42.8 7.7 8.9 86.1 1.0 4.4 5.4 5.7 4.0 2.0 0.4 3.5 53.3 4 1 -1.6',
 '7']

In [52]:
new_lines = lines
del new_lines[1::3]

In [65]:
lines[0:5]

['PLAYER TEAM AGE GP W L MIN PTS FGM FGA FG% 3PM 3PA 3P% FTM FTA FT% OREB DREB REB AST TOV STL BLK PF FP DD2 TD3 +/-',
 'Michael Jordan',
 'CHI 34 82 69 13 52.4 41.0 15.5 31.9 48.6 1.9 5.0 37.4 8.1 9.7 83.3 1.9 6.2 8.1 5.9 2.8 2.4 0.7 2.6 66.2 9 1 13.8',
 'Karl Malone',
 'UTA 33 82 64 18 51.9 39.0 15.0 27.2 55.0 0.0 0.2 0.0 9.0 12.0 75.5 3.3 10.7 14.0 6.4 4.0 2.0 0.8 3.7 69.7 43 1 13.3']

In [64]:
stats = new_lines[2::2] #.split(' ')
stats[0].split(' ')

['CHI',
 '34',
 '82',
 '69',
 '13',
 '52.4',
 '41.0',
 '15.5',
 '31.9',
 '48.6',
 '1.9',
 '5.0',
 '37.4',
 '8.1',
 '9.7',
 '83.3',
 '1.9',
 '6.2',
 '8.1',
 '5.9',
 '2.8',
 '2.4',
 '0.7',
 '2.6',
 '66.2',
 '9',
 '1',
 '13.8']

In [73]:
new_lines
index = new_lines[1::2]
print(len(index))
index[-1:-15:-1]

444


['438',
 '438',
 '438',
 '438',
 '438',
 '1 0 1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0 0.0',
 '438',
 'MIA 25 8 8 0 48.0 1.5 0.0 11.9 0.0 0.0 6.0 0.0 1.5 3.0 50.0 1.5 7.5 9.0 4.5 3.0 3.0 0.0 7.5 24.9 0 0 -17.9',
 'HOU 40 12 7 5 52.1 2.3 1.1 2.8 40.0 0.0 0.0 0.0 0.0 0.0 0.0 2.8 4.5 7.3 1.7 0.0 1.1 2.3 4.5 23.8 0 0 0.6',
 'SAS 33 10 2 8 50.6 3.4 0.9 4.3 20.0 0.0 0.0 0.0 1.7 1.7 100 2.6 4.3 6.8 1.7 6.0 0.9 0.9 3.4 13.3 0 0 -10.3',
 'IND 36 26 9 17 48.1 3.5 1.0 6.6 15.8 0.0 0.0 0.0 1.4 2.1 66.7 2.4 9.4 11.8 0.7 3.1 1.0 2.1 11.5 25.0 0 0 -15.0',
 'MIL 27 5 2 3 52.7 5.1 2.2 5.8 37.5 0.0 0.0 0.0 0.7 5.8 12.5 1.4 5.1 6.5 0.7 1.4 1.4 0.7 8.7 19.1 0 0 -2.2',
 'ATL 26 72 49 23 51.5 6.1 2.4 5.5 44.0 0.0 0.0 0.0 1.2 3.2 38.5 4.6 7.8 12.4 1.5 2.5 1.7 0.9 7.0 28.4 0 0 0.4',
 'BOS 30 6 1 5 50.5 6.1 2.4 6.5 37.5 0.0 0.0 0.0 1.2 2.0 60.0 3.7 6.9 10.6 1.6 2.8 2.0 3.7 8.5 35.4 0 0 3.3']

In [53]:
df = pd.DataFrame(new_lines[2::2])

['PLAYER TEAM AGE GP W L MIN PTS FGM FGA FG% 3PM 3PA 3P% FTM FTA FT% OREB DREB REB AST TOV STL BLK PF FP DD2 TD3 +/-',
 'Michael Jordan',
 'CHI 34 82 69 13 52.4 41.0 15.5 31.9 48.6 1.9 5.0 37.4 8.1 9.7 83.3 1.9 6.2 8.1 5.9 2.8 2.4 0.7 2.6 66.2 9 1 13.8',
 'Karl Malone',
 'UTA 33 82 64 18 51.9 39.0 15.0 27.2 55.0 0.0 0.2 0.0 9.0 12.0 75.5 3.3 10.7 14.0 6.4 4.0 2.0 0.8 3.7 69.7 43 1 13.3',
 'David Robinson',
 'SAS 31 6 3 3 52.5 37.9 12.9 25.7 50.0 0.0 0.0 0.0 12.1 18.6 65.4 6.8 11.4 18.2 2.9 2.9 2.1 2.1 3.2 74.0 2 0 7.5',
 "Shaquille O'Neal",
 'LAL 25 51 38 13 52.3 36.0 14.9 26.7 55.7 0.0 0.1 0.0 6.2 12.9 48.4 5.2 12.0 17.2 4.3 3.9 1.2 4.0 4.8 74.7 44 0 7.2',
 'Chris Gatling',
 'NJN 29 47 16 31 51.1 35.5 13.0 24.8 52.5 0.0 0.2 16.7 9.4 13.1 71.7 5.3 9.4 14.7 1.1 4.8 1.6 1.2 5.5 58.5 16 0 -3.5',
 'Mitch Richmond',
 'SAC 32 81 34 47 51.4 35.2 12.0 26.5 45.4 3.4 8.0 42.8 7.7 8.9 86.1 1.0 4.4 5.4 5.7 4.0 2.0 0.4 3.5 53.3 4 1 -1.6',
 'Glen Rice',
 'CHH 30 79 52 27 53.5 33.7 11.5 24.1 47.7 3.3

In [22]:
## Select ALL pages of data

xpath_page_selection = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'
xpath_all_pages = xpath_page_selection + '/option[1]'

browser.find_element_by_xpath(xpath_all_pages).click()

In [27]:
## scrape the stats table

xpath_stats_table = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]'
table = browser.find_element_by_xpath(xpath_stats_table)
lines = table.text.split('\n')
print(len(lines))
lines[0:10] 


1588


['PLAYER TEAM AGE GP W L MIN OFFRTG DEFRTG NETRTG AST% AST/TO AST RATIO OREB% DREB% REB% TO RATIO EFG% TS% USG% PACE PIE',
 '1',
 'Maxi Kleber',
 'DAL 28 74 42 32 25.5 112.6 109.2 3.4 6.5 1.52 12.6 5.4 13.6 9.5 8.3 57.7 60.5 13.4 101.50 8.2',
 '2',
 'Delon Wright',
 'DAL 28 73 42 31 21.5 111.2 109.5 1.7 21.5 3.34 31.6 4.0 12.4 8.2 9.4 51.9 55.4 14.0 101.55 10.6',
 '2',
 'Dillon Brooks',
 'MEM 24 73 34 39 28.9 108.5 108.9 -0.3 10.4 1.21 10.6 3.0 7.6 5.4 8.7 47.6 51.0 24.7 103.55 6.2']

In [32]:
## The types of data to click into and scrape

stat_types = ["Traditional", "Advanced", "Misc", "Scoring", "Usage", "Opponent", "Defense"]

## select type button to get drop down menu

xpath_stat_type_button = '/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/div/a'
browser.find_element_by_xpath(xpath_stat_type_button).click()

## select different type
xpath_traditional = '/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/ul/li[4]/a/span'
browser.find_element_by_xpath(xpath_traditional).click()
